In [4]:
# Calculate monthly volumes from hyperbolic arps parameters

import numpy as np
import pandas as pd

# Arps inputs
Qi = 800 # Input as daily rate
Dei = 0.639 # Input as annual effective decline
Def = 0.06 # Input as annual effective decline
b = 1.4
months = 600
time = []
d = []
rate = []
vol = []

# Calculations to determine decline type
if Dei == Def:
    Type = 'exp'
elif Dei > Def and b == 1:
    Type = 'har'
    Dn = Dei / (1 - Dei)
else:
    Type = 'hyp'
    Dn = (1 / b) * (((1 - Dei) ** -b) - 1)

# Generate volumes
for t in range(months):      
    if Type == 'hyp':
        Dn_t = Dn / (1 + b * Dn * (t / 12))
        De_t = 1 - (1 / ((Dn_t * b) + 1)) ** (1 / b)
        if De_t > Def:
            q = Qi * (1 + b * Dn * (t / 12)) ** (-1/b)
            Np = ((Qi ** b) / (Dn * (1 - b))) * ((Qi ** (1 - b)) - (q ** (1 - b))) * 365
        else:
            q = rate[t - 1] * np.exp(-(-np.log(1 - Def)) * (1 / 12))
            Np = ((rate[t - 1] - q) / (-np.log(1 - Def)) * 365) + vol[t - 1]
            De_t = Def
        time.append(t)
        d.append(De_t)
        rate.append(q)
        vol.append(Np)
    elif Type == 'har':
        Dn_t = Dn / (1 + Dn * (t / 12))
        De_t = 1 - (1 / (Dn_t + 1))
        if De_t > Def:
            q = Qi / (1 + b * Dn * (t / 12))
            Np = (Qi / Dn) * np.log(Qi / q) * 365
        else:
            q = rate[t - 1] * np.exp(-(-np.log(1 - Def)) * (1 / 12))
            Np = ((rate[t - 1] - q) / (-np.log(1 - Def)) * 365) + vol[t - 1]
            De_t = Def
        time.append(t)
        d.append(De_t)
        rate.append(q)
        vol.append(Np)
    else:
        q = Qi * np.exp(-(-np.log(1 - Dei)) * (t / 12))
        Np = (Qi - q) / (-np.log(1 - Dei)) * 365
        De_t = Dei
        time.append(t)
        d.append(De_t)
        rate.append(q)
        vol.append(Np)

# Create and format pandas data frame
Out = pd.DataFrame({'Month':time, 'De':d, 'Rate':rate, 'Np':vol}, index = time)
Out['De'] = Out['De'].map('{:.4f}'.format)
Out['Rate'] = Out['Rate'].map('{:,.2f}'.format)
Out['Np'] = Out['Np'].map('{:,.2f}'.format)

# Calculate EUR
EUR = max(vol)
print(f'The EUR for a life of {months / 12:,.2f} years is {EUR:,.2f}')

# Output monthly volumes to csv
Out.to_csv('monthly.csv')

The EUR for a life of 50.00 years is 908,498.17
